In [ ]:
def train_sent(read_f, write_f, length):
    with open(read_f, 'r') as f:
        count = 0
        for sent in f:
            if count >= length:
                break
            with open(write_f, 'a+') as w:
                if len(sent) > 1:
                    w.write(sent)
                    count += 1

In [ ]:
def txt_write_csv(csv_file, txt_file):
    import pandas as pd
    sent = []
    label = []
    with open(txt_file, 'r') as f:
        for line in f:
            sent.append(line)
            label.append('n')
    dataframe = pd.DataFrame({'sentence':sent, 'label':label})
    dataframe.to_csv(csv_file, index=False, sep=',')

In [2]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
def sent_vector(csv_file, length):
    if length > 0:
        csv_data = pd.read_csv(csv_file, low_memory = False, nrows=length)#防止弹出警告
    else:
        csv_data = pd.read_csv(csv_file, low_memory = False)
    csv_df = pd.DataFrame(csv_data)
    sentences = csv_df['sentence']
    label = csv_df['label']
    label.loc[label == 'n'] = 0
    label.loc[label == 'p'] = 1
    model = word2vec.Word2Vec.load('Model/n_cbow.w2v')
    size = model.wv.syn0.shape[1]
    sents_vec = []
    for sent in sentences:
        sents_vec.append(compute_sent_vec(sent, model, size))
    return np.array(sents_vec), label, sentences
        

In [2]:
def compute_sent_vec(sent, model, size):
    vec = np.zeros(size)
    for word in sent.split():
        if word in model.wv:
            vec += model.wv[word]
    vec /= len(sent.split())
    return vec


In [3]:
sents_vec, label, sentences = sent_vector('Data/training_data.csv', -1)

/workspace/anaconda3/envs/acai/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/workspace/anaconda3/envs/acai/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  from ipykernel import kernelapp as app


In [ ]:
#self implemented logistic regression
from logistic_regression import LogisticRegression
l = LogisticRegression(sents_vec[:700], label[:700])
cost = l.train(0.1,1000)
pred_y = l.predict(sents_vec[300:], label[300:])

In [ ]:
#sklearn logistic regression result
from sklearn.linear_model import LogisticRegression
label = np.array(label, dtype='int')
clf = LogisticRegression(max_iter=1000).fit(sents_vec[:700], label[:700])
test_accuracy = clf.score(sents_vec[300:], label[300:])*100
pred = clf.predict(sents_vec[300:])
print("正确率为%s%%" % test_accuracy)

In [58]:
class kMeans:
    def train(self, m, sents_vec, sentences):
        centroids = []
        for i in range(m):
            centroids.append(sents_vec[i])
        while True:
            clusters, sents_cluster = self.choose_cluster(m, sents_vec, centroids, sentences)
            n_centroids = self.update_centroids(clusters)
            flag = 0
            for i in range(len(centroids)):
                if i < len(n_centroids):
                    if not all(centroids[i] == n_centroids[i]):
                        flag = 1
                else:
                    flag = 1
            if flag:
                centroids = n_centroids
            else:
                break
        return centroids, sents_cluster

    def choose_cluster(self, m, sents_vec, centroids, sentences):
        clusters = {}
        sents_cluster = {}
        for j in range(sents_vec.shape[0]):
            vec = sents_vec[j]
            min_dis = 2
            min_c = -1
            if any(vec) != 0:
                for i in range(len(centroids)):
                    sim = 1-self.cosine_simi(vec, centroids[i])
                    if sim < min_dis:
                        min_dis = sim
                        min_c = i
                if min_c not in clusters:
                    clusters[min_c] = []
                    sents_cluster[min_c] = []
                clusters[min_c].append(vec)
                sents_cluster[min_c].append(sentences[j])
        return clusters, sents_cluster

    def update_centroids(self, clusters):
        centroids = []
        for k, s_vec in clusters.items():
            c = np.zeros(s_vec[0].shape[0])
            for vec in s_vec:
                c = c + vec
            if all(c) != 0:
                c = c / len(s_vec)
                centroids.append(c)
        return centroids

    def cosine_simi(self, vec, cent):
        vec_norm = np.sqrt(np.dot(vec, vec))
        cent_norm = np.sqrt(np.dot(cent, cent))
        return np.dot(vec, cent) / (vec_norm * cent_norm)

In [59]:
#self implemented k means
#from k_means import kMeans
km = kMeans()
centroids, sents_cluster = km.train(4, sents_vec[:10000], sentences[:10000])

In [1]:
sents_cluster[1]

NameError: name 'sents_cluster' is not defined